# Train a regressor for the locations of instruments

In [ ]:
import os
os.chdir('/home/extra/micheal/pixel2style2pixel')

## Design lables

- 0: the presence of instruments
- 1: x coordinate of instruments
- 2: y coordinate of instruments
- 3: x span of instruments
- 4: y span of instruments
- 5: the presence of shadows
- 6: x coordinate of shadows
- 7: y coordinate of shadows
- 8: x span of shadows
- 9: y span of shadows

## Generate regressor labels from the segmentation

Note: the coordinates are **not** normalized

In [ ]:
from utils.regressor_utils import get_coor_avg, attribute_label_from_segmentation

## Define a regressor

In [ ]:
from models.regressor import Regressor, LightningRegressor

regressor = Regressor()

Test regressor

In [ ]:
import torch

x = torch.randn((4, 1, 256, 256))
with torch.no_grad():
    output = regressor(x)
output.shape

## Train the regressor

In [ ]:
from argparse import Namespace
import time
import sys
import pprint
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from dataset import augmentations
from utils.common import tensor2im, log_input_image
from models.psp import pSp

# added imports
import os
import imageio
import matplotlib.pyplot as plt
from configs.transforms_config import SegToImageTransforms
from glob import glob
from training.coach import Coach


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
model_path = '/home/extra/micheal/pixel2style2pixel/experiments/ioct_seg2bscan2/checkpoints/best_model.pt'

In [ ]:
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
optss = Namespace(**opts)
optss.batch_size = 8
optss.stylegan_weights = model_path
optss.load_partial_weights = True

coach = Coach(optss)

In [ ]:
train_loader = iter(coach.train_dataloader)
batch = next(train_loader)
test_label = batch[0].float()
test_bscan = batch[1].float()
print('label shape', test_label.shape)
print('bscan shape', test_bscan.shape)

In [ ]:
# model
regressor = LightningRegressor(Regressor())

In [ ]:
# train model
trainer = pl.Trainer(gpus=1, max_epochs=100, weights_save_path="artifacts/weights/")
trainer.fit(model=regressor, train_dataloaders=coach.train_dataloader, )

In [ ]:
torch.save(regressor.regressor.state_dict(), "artifacts/weights/regressor.pt")

In [ ]:
torch.save(regressor.state_dict(), "artifacts/weights/regressor_pl.pt")